# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [3]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [4]:
data = pd.read_csv("kddcup.data_10_percent", header=None, names = col_names)

# 前処理
## カテゴリ化

In [5]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [6]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [7]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [8]:
data['label3'] = data.label.copy()

In [9]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop'),'label3'] = 'DoS'

In [10]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit'),'label3'] = 'U2R'

In [11]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster'),'label3'] = 'R2L'

In [12]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan'),'label3'] = 'Probe'

In [13]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

## サンプリング

In [14]:
data = resample(data,n_samples=5000,random_state=0)

In [15]:
data.shape

(5000, 44)

## 数値化

In [16]:
le_protocol_type = preprocessing.LabelEncoder()

In [17]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [18]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [19]:
le_service = preprocessing.LabelEncoder()

In [20]:
le_service.fit(data.service)

LabelEncoder()

In [21]:
data.service = le_service.transform(data.service)

In [22]:
le_flag = preprocessing.LabelEncoder()

In [23]:
le_flag.fit(data.flag)

LabelEncoder()

In [24]:
data.flag = le_flag.transform(data.flag)

In [25]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.0,5000.000000,5000.0,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,52.523800,0.470600,16.893000,4.558800,1.918734e+03,504.657000,0.0,0.010400,0.0,0.035000,...,231.721200,189.682600,0.759340,0.031328,0.604468,0.007958,0.173314,0.172940,0.055440,0.05416
std,652.490873,0.581724,9.551995,1.144205,7.269032e+04,3853.303587,0.0,0.175208,0.0,0.774012,...,65.810307,105.291222,0.407655,0.112308,0.480702,0.051469,0.377746,0.377929,0.224688,0.22331
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,10.000000,5.000000,4.800000e+01,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,66.000000,0.530000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,10.000000,5.000000,5.200000e+02,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,0.000000,1.000000,29.000000,5.000000,1.032000e+03,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.030000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,17779.000000,2.000000,47.000000,6.000000,5.133876e+06,202976.000000,0.0,3.000000,0.0,28.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [26]:
data.shape

(5000, 44)

## ラベルの分離

In [27]:
y_train_1 = data.label.copy() 

In [28]:
y_train_2 = data.label2.copy()

In [29]:
y_train_3 = data.label3.copy()

In [30]:
x_train = data.drop(['label','label2','label3'],axis=1)

In [31]:
x_train.shape

(5000, 41)

In [32]:
y_train_1.shape

(5000,)

In [33]:
y_train_2.shape

(5000,)

In [34]:
y_train_3.shape

(5000,)

## 標準化

In [35]:
ss = preprocessing.StandardScaler()

In [36]:
ss.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [37]:
x_train = ss.transform(x_train)

In [38]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [39]:
pd.DataFrame(x_train,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.0,5.000000e+03,5000.0,5.000000e+03,...,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03
mean,-8.712836e-16,1.166178e-16,-1.456946e-16,3.965717e-16,-1.608966e-16,8.072432e-16,0.0,-1.102429e-15,0.0,-2.287948e-15,...,-4.490852e-16,-4.384049e-16,-1.938893e-16,-1.633416e-16,-1.923794e-16,-2.957301e-16,-1.285194e-16,-1.654676e-16,3.603284e-16,-2.677525e-16
std,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,0.0,1.000100e+00,0.0,1.000100e+00,...,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00
min,-8.050542e-02,-8.090550e-01,-1.768708e+00,-3.984651e+00,-2.639865e-02,-1.309805e-01,0.0,-5.936383e-02,0.0,-4.522345e-02,...,-3.506203e+00,-1.792186e+00,-1.862889e+00,-2.789752e-01,-1.257595e+00,-1.546327e-01,-4.588568e-01,-4.576449e-01,-2.467673e-01,-2.425569e-01
25%,-8.050542e-02,-8.090550e-01,-7.217016e-01,3.856339e-01,-2.573824e-02,-1.309805e-01,0.0,-5.936383e-02,0.0,-4.522345e-02,...,3.537611e-01,-1.174789e+00,-5.626398e-01,-2.789752e-01,-1.257595e+00,-1.546327e-01,-4.588568e-01,-4.576449e-01,-2.467673e-01,-2.425569e-01
50%,-8.050542e-02,-8.090550e-01,-7.217016e-01,3.856339e-01,-1.924430e-02,-1.309805e-01,0.0,-5.936383e-02,0.0,-4.522345e-02,...,3.537611e-01,6.204120e-01,5.904111e-01,-2.789752e-01,8.229036e-01,-1.546327e-01,-4.588568e-01,-4.576449e-01,-2.467673e-01,-2.425569e-01
75%,-8.050542e-02,9.101439e-01,1.267611e+00,3.856339e-01,-1.220001e-02,-1.309805e-01,0.0,-5.936383e-02,0.0,-4.522345e-02,...,3.537611e-01,6.204120e-01,5.904111e-01,-1.182581e-02,8.229036e-01,-1.546327e-01,-4.588568e-01,-4.576449e-01,-2.467673e-01,-2.425569e-01
max,2.717011e+01,2.629343e+00,3.152222e+00,1.259691e+00,7.060734e+01,5.255013e+01,0.0,1.706482e+01,0.0,3.613354e+01,...,3.537611e-01,6.204120e-01,5.904111e-01,8.626003e+00,8.229036e-01,1.927647e+01,2.188689e+00,2.188619e+00,4.204302e+00,4.235968e+00


## 学習

In [40]:
pca = PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [41]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [42]:
x_train2 = pca.transform(x_train)

In [43]:
x_train2.shape

(5000, 3)

In [44]:
clf = SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1.0, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
t1=time.perf_counter()
clf.fit(x_train2,y_train_2)
t2=time.perf_counter()

In [46]:
print(t2-t1,"秒")

38.69440309900165 秒


## 予測

In [47]:
t1=time.perf_counter()
pred=clf.predict(x_train2)
t2=time.perf_counter()


In [48]:
print(t2-t1,"秒")

0.0161432809982216 秒


In [49]:
print(classification_report(y_train_2, pred))
print(confusion_matrix(y_train_2, pred))

             precision    recall  f1-score   support

      atack       1.00      0.99      0.99      3985
    normal.       0.94      0.99      0.97      1015

avg / total       0.99      0.99      0.99      5000

[[3926   59]
 [   6 1009]]


## 学習

In [50]:
pca2 = PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [51]:
clf2 = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.17782794100389229,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [52]:
pca2.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [53]:
x_train3 = pca.transform(x_train)

In [54]:
t1=time.perf_counter()
clf2.fit(x_train3,y_train_3)
t2=time.perf_counter()

In [55]:
print(t2-t1,"秒")

0.05409338200115599 秒


## 予測

In [56]:
t1=time.perf_counter()
pred2=clf2.predict(x_train3)
t2=time.perf_counter()


In [57]:
print(t2-t1,"秒")

0.02707723999992595 秒


In [58]:
print(classification_report(y_train_3, pred2))
print(confusion_matrix(y_train_3, pred2))

             precision    recall  f1-score   support

        DoS       1.00      0.99      1.00      3928
      Probe       1.00      0.80      0.89        44
        R2L       1.00      0.08      0.15        12
        U2R       1.00      1.00      1.00         1
    normal.       0.97      1.00      0.98      1015

avg / total       0.99      0.99      0.99      5000

[[3904    0    0    0   24]
 [   5   35    0    0    4]
 [   3    0    1    0    8]
 [   0    0    0    1    0]
 [   3    0    0    0 1012]]
